### Loading Packages

In [1]:
library(dplyr)

current_date <- as.Date("2025-02-05")



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
cbbdata::cbd_login(username = "Naki", password = "Duke@123")
data <- cbbdata::cbd_torvik_team_factors(year = 2025, no_bias = TRUE) 
print(data)

API Key set!# A tibble: 364 x 31
   team   conf  games  wins losses adj_t adj_o adj_d barthag   efg def_efg   ftr
   <chr>  <chr> <dbl> <dbl>  <dbl> <dbl> <dbl> <dbl>   <dbl> <dbl>   <dbl> <dbl>
 1 Houst~ B12      21    17      4  62.4  124.  85.3   0.987  52.7    43.4  27  
 2 Auburn SEC      21    20      1  69.2  133.  93.7   0.982  56.4    44.7  30.5
 3 Duke   ACC      21    19      2  66.6  123.  88.4   0.978  56.3    43.6  32.7
 4 Tenne~ SEC      22    18      4  65.1  115.  86.7   0.963  51.7    42    31.6
 5 Alaba~ SEC      22    19      3  75.3  126.  95.9   0.959  55.6    45.9  42.5
 6 Flori~ SEC      22    19      3  70    123.  94.2   0.956  53.6    43.7  32  
 7 Illin~ B10      22    15      7  73.4  119.  92.1   0.952  51.8    44    36.4
 8 Kansas B12      22    16      6  69.4  117.  90.2   0.950  53.4    43.8  23.5
 9 Texas~ B12      21    17      4  66.4  124.  96.3   0.948  56.7    47.1  30.7
10 Purdue B10      23    18      5  66.2  123.  96.5   0.943  56.8    50.6  

## Extract ACC and B10 Teams from list

In [3]:
# # Filter teams in ACC and B10 and extract into separate lists
acc_teams <- data |>
  filter(conf == "ACC") |>
  pull(team) # Extract the 'team' column as a list

# b10_teams <- data |>
#   filter(conf == "B10") |>
#   pull(team) # Extract the 'team' column as a list

# SEC_teams <- data |>
#   filter(conf == "SEC") |>
#   pull(team) # Extract the 'team' column as a list

# misc_games <- c(
#   "Tennessee", "Auburn", "Iowa St.", "Duke", "Alabama", "Florida", 
#   "Kansas", "Marquette", "Oregon", "Kentucky", "Connecticut", "Oklahoma", 
#   "Texas A&M", "Gonzaga", "Houston", "Mississippi", "Cincinnati", 
#   "Michigan St.", "Mississippi St.", "San Diego St.", 
#   "Purdue", "UCLA", "Arkansas", "Illinois", "Baylor", "Colorado St.", "Georgia St.", 
#   "North Carolina Central", "Iowa St.", 
#   "Richmond", "Boise St.", "Nevada", "Eastern Kentucky",  "Iona", "Texas St.", 
#   "Army", "Northeastern", "Northwestern"
# )

#unique_teams <- unique(c(acc_teams, b10_teams, SEC_teams))
unique_teams <- c("Duke", "North Carolina", "North Carolina St.")

## Extract Relevant Factors

In [4]:
selected_data_columns <- data |> 
  dplyr::select(Team = team, 
                ADJO = adj_o, 
                ADJD = adj_d,
                EFG_pct = efg, 
                TO_Pct = tov_rate, 
                OR_Pct = oreb_rate,
                FTR_Pct = ftr, 
                ThreePointer_Pct = three_pt_pct)

print(selected_data_columns)
print(which(is.na(selected_data_columns), arr.ind = TRUE))

# A tibble: 364 x 8
   Team        ADJO  ADJD EFG_pct TO_Pct OR_Pct FTR_Pct ThreePointer_Pct
   <chr>      <dbl> <dbl>   <dbl>  <dbl>  <dbl>   <dbl>            <dbl>
 1 Houston     124.  85.3    52.7   14.2   38      27               39.3
 2 Auburn      133.  93.7    56.4   13.8   36.7    30.5             37.7
 3 Duke        123.  88.4    56.3   15.4   34.9    32.7             36.8
 4 Tennessee   115.  86.7    51.7   16.3   38.4    31.6             33.2
 5 Alabama     126.  95.9    55.6   17.3   36.7    42.5             33.6
 6 Florida     123.  94.2    53.6   15.7   40.2    32               33.9
 7 Illinois    119.  92.1    51.8   16.2   36.7    36.4             31.1
 8 Kansas      117.  90.2    53.4   16.1   31.1    23.5             34.5
 9 Texas Tech  124.  96.3    56.7   16.1   36.4    30.7             38.2
10 Purdue      123.  96.5    56.8   16.1   30.2    37.7             38.3
# i 354 more rows
     row col


## Cleaning Season Schedule

In [5]:
all_games <- cbbdata::cbd_torvik_season_schedule(year = 2025)
all_games <- all_games |>
  mutate(Location = ifelse(neutral, "N", "H")) |>
  select(-neutral, -type, -year, -game_id) |>
  rename(Team = home,
         Opponent = away,
         )

print(which(is.na(all_games), arr.ind = TRUE))
print(all_games)

     row col
            date              Team         Opponent Location
          <Date>            <char>           <char>   <char>
   1: 2024-11-04             Tulsa Northeastern St.        H
   2: 2024-11-04         Tennessee     Gardner Webb        H
   3: 2024-11-04 Houston Christian            Avila        H
   4: 2024-11-04         Stonehill   Framingham St.        H
   5: 2024-11-04    North Carolina             Elon        H
  ---                                                       
5861: 2025-03-09          Nebraska             Iowa        H
5862: 2025-03-09  Florida Atlantic    East Carolina        H
5863: 2025-03-09            Temple      North Texas        H
5864: 2025-03-09       Wichita St.            Tulsa        H
5865: 2025-03-09        Washington           Oregon        H


## Finding Games for Particluar Data

In [6]:
upcoming_games <- all_games |>
  filter(date >= current_date & date <= as.Date("2025-02-14"))
print(upcoming_games)

required_upcoming_games <- upcoming_games |>
  filter(Team %in% unique_teams | Opponent %in% unique_teams)
print(required_upcoming_games)

           date          Team          Opponent Location
         <Date>        <char>            <char>   <char>
  1: 2025-02-05       Memphis             Tulsa        H
  2: 2025-02-05     Tennessee          Missouri        H
  3: 2025-02-05       Samford            Mercer        H
  4: 2025-02-05 James Madison              Troy        H
  5: 2025-02-05          UTSA            Tulane        H
 ---                                                    
454: 2025-02-14     Dartmouth          Columbia        H
455: 2025-02-14     Manhattan         Merrimack        H
456: 2025-02-14       Indiana              UCLA        H
457: 2025-02-14          Iona             Rider        H
458: 2025-02-14     Green Bay Northern Kentucky        H
         date               Team           Opponent Location
       <Date>             <char>             <char>   <char>
1: 2025-02-05           Syracuse               Duke        H
2: 2025-02-05         California North Carolina St.        H
3: 2025-02-08  

In [7]:
selected_data_columns$Team <- gsub("N\\.C\\. State", "North Carolina St.", selected_data_columns$Team)


# Extract stats for Team and Opponent
team_data <- required_upcoming_games |>
  left_join(selected_data_columns, by = c("Team" = "Team"))

opponent_data <- required_upcoming_games |>
  left_join(selected_data_columns, by = c("Opponent" = "Team")) |>
  rename_with(~ paste("Opp_", ., sep = ""), everything())

# Rename and remove unwanted columns
opponent_data <- opponent_data |>
  rename(Opponent = Opp_Opponent, Team = Opp_Team) |>
  select(-c("Opp_date", "Opp_Location"))

# Combine both datasets
final_data <- team_data |>
  left_join(opponent_data, by = c("Team", "Opponent"))

# Print the final result
print(final_data)

setdiff(required_upcoming_games$Team, selected_data_columns$Team)
setdiff(required_upcoming_games$Opponent, selected_data_columns$Team)


         date               Team           Opponent Location     ADJO      ADJD
       <Date>             <char>             <char>   <char>    <num>     <num>
1: 2025-02-05           Syracuse               Duke        H 107.7863 105.76595
2: 2025-02-05         California North Carolina St.        H 110.5322 107.61618
3: 2025-02-08            Clemson               Duke        H 116.9055  97.67072
4: 2025-02-08           Stanford North Carolina St.        H 113.7652 102.50955
5: 2025-02-08     North Carolina         Pittsburgh        H 115.4657  99.69210
6: 2025-02-10            Clemson     North Carolina        H 116.9055  97.67072
7: 2025-02-12               Duke         California        H 122.7361  88.35591
8: 2025-02-12 North Carolina St.         Louisville        H 108.6292 101.08028
   EFG_pct TO_Pct OR_Pct FTR_Pct ThreePointer_Pct Opp_ADJO  Opp_ADJD
     <num>  <num>  <num>   <num>            <num>    <num>     <num>
1:    50.7   17.7   29.8    33.3             31.9 122.7361  88

character(0)

character(0)

In [8]:
file_to_write <- "ACC_test_file_from_5thFeb_14thFeb.csv"
write.csv(final_data, file_to_write, row.names = FALSE)